In [1]:
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
path = "D:\Algorithmic Marketing\coupon-redemption-smote-feature-selection"
csv_pattern = os.path.join(path, "*.csv")
file_list = glob.glob(csv_pattern)

print("Created dataframe names: ")
for file in file_list:
    
    vars()[os.path.basename(file)[:-4]] = pd.read_csv(file)
    print(os.path.basename(file)[:-4])
  

Created dataframe names: 
campaign_data
coupon_item_mapping
customer_demographics
customer_transaction_data
final_train
item_data
smote
test
train


In [3]:
print(customer_transaction_data.isnull().sum())

date               0
customer_id        0
item_id            0
quantity           0
selling_price      0
other_discount     0
coupon_discount    0
dtype: int64


In [24]:
print(campaign_data.shape)
customer_transaction_data[customer_transaction_data.coupon_discount!=0].shape

(28, 4)


(21286, 7)

In [39]:
df = pd.merge(campaign_data, train, on="campaign_id")
df = df["campaign_id"].groupby(["campaign_id", "redemeption_status"]).count()
df

KeyError: 'campaign_id'

In [4]:
customer_transaction_data.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


In [5]:
customer_transaction_data[["customer_id", "item_id", "quantity"]].head()

,customer_id,item_id,quantity
0,1501,26830,1
1,1501,54253,1
2,1501,31962,1
3,1501,33647,1
4,1501,48199,1


In [ ]:
# Using implicit package from Cython 

import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

from sklearn.preprocessing import MinMaxScaler

import implicit

# Load the data in simialr fashion and keep the required columns, like User , MCAT and PURCHASE count
#raw_data = pd.read_csv('cat_13.csv')
#raw_data = raw_data.loc[:,['GLUSR','MAPPED_MCAT','PURCHASE_CNT']]
raw_data = customer_transaction_data[["customer_id", "item_id", "quantity"]]
#raw_data = raw_data.drop(raw_data.columns[1], axis=1)
#raw_data.columns = ['user', 'mcat', 'purchase_cnt']
raw_data.columns = ['user', 'items', 'quantity']

# Drop missing values from the data
data = raw_data.dropna()
data = data.copy()

# Create a categorical user_id and mcat_id column
data['user'] = data['user'].astype("category")
data['mcat'] = data['mcat'].astype("category")
data['user_id'] = data['user'].cat.codes
data['mcat_id'] = data['mcat'].cat.codes

# The implicit library expects data as a item-user matrix so we need to create two matricies, one for fitting the model (item-user) 
# and another for recommendations (user-item)
sparse_item_user = sparse.csr_matrix((data['purchase_cnt'].astype(float), (data['mcat_id'], data['user_id'])))
sparse_user_item = sparse.csr_matrix((data['purchase_cnt'].astype(float), (data['user_id'], data['mcat_id'])))

# Initializing the als algoritm and train the model using item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

# Calculating the confidence by multiplying sparse matrix with the alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

#Fitting the model
model.fit(data_conf)

Ids = list(data.user_id.unique())
R1 = pd.DataFrame(columns=['mcat_id','score','user_id'])
R1


for i in range(len(Ids)):
    user_id = Ids[i]
    
    recommended = model.recommend(user_id, sparse_user_item)

    mcats = []
    scores = []
    users = []

# Get mcat names from mcat ids
    for item in recommended:
        idx, score = item
        mcats.append(data.mcat.loc[data.mcat_id == idx].iloc[0])
        #users.append(data.user.loc[data.user_id== idx].iloc[0])
        scores.append(score)

    # Create a dataframe of mcat names and scores
    recommendations = pd.DataFrame({'mcat_id': mcats, 'score': scores})
    recommendations['user_id'] = data.user.loc[data.user_id== Ids[i]].iloc[0]

#print(recommendations)
    
    R1 = R1.append(recommendations)
        
        
print(R1.shape)

#Saving the outputs in the required format
R2 = R1[['user_id','mcat_id','score']]
R2 = R2.sort_values(by='user_id')
R2.to_excel('user_to_mcat_recommen_ALS_grp_id13.xlsx',index=False)


In [6]:
# Using implicit package from Cython 

import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

from sklearn.preprocessing import MinMaxScaler

import implicit

# Load the data in simialr fashion and keep the required columns, like User , MCAT and PURCHASE count
#raw_data = pd.read_csv('cat_13.csv')
#raw_data = raw_data.loc[:,['GLUSR','MAPPED_MCAT','PURCHASE_CNT']]
raw_data = customer_transaction_data[["customer_id", "item_id", "quantity"]]
#raw_data = raw_data.drop(raw_data.columns[1], axis=1)
#raw_data.columns = ['user', 'mcat', 'purchase_cnt']
raw_data.columns = ['user', 'items', 'quantity']

# Drop missing values from the data
data = raw_data.dropna()
data = data.copy()
print(data.head(10))




# The implicit library expects data as a item-user matrix so we need to create two matricies, one for fitting the model (item-user) 
# and another for recommendations (user-item)
sparse_item_user = sparse.csr_matrix((data['quantity'].astype(float), (data['items'], data['user'])))
sparse_user_item = sparse.csr_matrix((data['quantity'].astype(float), (data['user'], data['items'])))
sparse_item_user, sparse_item_user

   user  items  quantity
0  1501  26830         1
1  1501  54253         1
2  1501  31962         1
3  1501  33647         1
4  1501  48199         1
5  1501  57397         1
6   857  12424         1
7   857  14930         1
8   857  16657         1
9    67  10537         3


(<74067x1583 sparse matrix of type '<class 'numpy.float64'>'
 	with 787687 stored elements in Compressed Sparse Row format>,
 <74067x1583 sparse matrix of type '<class 'numpy.float64'>'
 	with 787687 stored elements in Compressed Sparse Row format>)

In [9]:
# Initializing the als algoritm and train the model using item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

In [10]:
# Calculating the confidence by multiplying sparse matrix with the alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')
#Fitting the model
model.fit(data_conf)

In [11]:
Ids = list(data.user.unique())
R1 = pd.DataFrame(columns=['items','score','user'])
R1

,items,score,user


In [12]:
for i in range(len(Ids)):
    user_id = Ids[i]
    recommended = model.recommend(user_id, sparse_user_item)
    items = []
    scores = []
    users = []
# Get mcat names from mcat ids
    for item in recommended:
        idx, score = item
        items.append(data.items.loc[data.items == idx].iloc[0])
        #users.append(data.user.loc[data.user_id== idx].iloc[0])
        scores.append(score)
    # Create a dataframe of mcat names and scores
    recommendations = pd.DataFrame({'items': items, 'score': scores})
    recommendations['user'] = data.user.loc[data.user== Ids[i]].iloc[0]
#print(recommendations)
    R1 = R1.append(recommendations)
print(R1.shape)

AttributeError: 'function' object has no attribute 'loc'

In [14]:
# Using implicit package from Cython
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
from sklearn.preprocessing import MinMaxScaler
import implicit
# Load the data in simialr fashion and keep the required columns, like User , MCAT and PURCHASE count
#raw_data = pd.read_csv('cat_13.csv')
#raw_data = raw_data.loc[:,['GLUSR','MAPPED_MCAT','PURCHASE_CNT']]
raw_data = customer_transaction_data[["customer_id", "item_id", "quantity"]]
#raw_data = raw_data.drop(raw_data.columns[1], axis=1)
#raw_data.columns = ['user', 'mcat', 'purchase_cnt']
raw_data.columns = ['user', 'items', 'quantity']
# Drop missing values from the data
data = raw_data.dropna()
data = data.copy()
print(data.head(10))
# Create a categorical user_id and mcat_id column
data['user'] = data['user'].astype("category")
data['items'] = data['items'].astype("category")
data['user_id'] = data['user'].cat.codes
data['item_id'] = data['items'].cat.codes
# The implicit library expects data as a item-user matrix so we need to create two matricies, one for fitting the model (item-user)
# and another for recommendations (user-item)
sparse_item_user = sparse.csr_matrix((data['quantity'].astype(float), (data['item_id'], data['user_id'])))
sparse_user_item = sparse.csr_matrix((data['quantity'].astype(float), (data['user_id'], data['item_id'])))
sparse_item_user, sparse_item_user
# Initializing the als algoritm and train the model using item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)
# Calculating the confidence by multiplying sparse matrix with the alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')
#Fitting the model
model.fit(data_conf)
Ids = list(data.user.unique())
R1 = pd.DataFrame(columns=['item_id','score','user_id'])
R1
for i in range(len(Ids)):
    user_id = Ids[i]
    recommended = model.recommend(user_id, sparse_user_item)
    items = []
    scores = []
    users = []
# Get mcat names from mcat ids
    for item in recommended:
        idx, score = item
        items.append(data['items'].loc[data.item_id == idx].iloc[0])
        #users.append(data.user.loc[data.user_id== idx].iloc[0])
        scores.append(score)
    # Create a dataframe of mcat names and scores
    recommendations = pd.DataFrame({'item_id': items, 'score': scores})
    recommendations['user_id'] = data.user.loc[data.user_id == Ids[i]].iloc[0]
#print(recommendations)
    R1 = R1.append(recommendations)
print(R1.shape)

   user  items  quantity
0  1501  26830         1
1  1501  54253         1
2  1501  31962         1
3  1501  33647         1
4  1501  48199         1
5  1501  57397         1
6   857  12424         1
7   857  14930         1
8   857  16657         1
9    67  10537         3


IndexError: index 1582 is out of bounds for axis 0 with size 1582

In [18]:
len(data.user.unique())

1582

In [ ]:
# Using implicit package from Cython 

import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

from sklearn.preprocessing import MinMaxScaler

import implicit

# Load the data in simialr fashion and keep the required columns, like User , MCAT and PURCHASE count
#raw_data = pd.read_csv('cat_13.csv')
#raw_data = raw_data.loc[:,['GLUSR','MAPPED_MCAT','PURCHASE_CNT']]
raw_data = customer_transaction_data[["customer_id", "item_id", "quantity"]] 
#raw_data = raw_data.drop(raw_data.columns[1], axis=1)
raw_data.columns = ['user', 'mcat', 'purchase_cnt']

# Drop missing values from the data
data = raw_data.dropna()
data = data.copy()

# Create a categorical user_id and mcat_id column
data['user'] = data['user'].astype("category")
data['mcat'] = data['mcat'].astype("category")
data['user_id'] = data['user'].cat.codes
data['mcat_id'] = data['mcat'].cat.codes

# The implicit library expects data as a item-user matrix so we need to create two matricies, one for fitting the model (item-user) 
# and another for recommendations (user-item)
sparse_item_user = sparse.csr_matrix((data['purchase_cnt'].astype(float), (data['mcat_id'], data['user_id'])))
sparse_user_item = sparse.csr_matrix((data['purchase_cnt'].astype(float), (data['user_id'], data['mcat_id'])))

# Initializing the als algoritm and train the model using item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

# Calculating the confidence by multiplying sparse matrix with the alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

#Fitting the model
model.fit(data_conf)

Ids = list(data.user_id.unique())
R1 = pd.DataFrame(columns=['mcat_id','score','user_id'])
R1


for i in range(len(Ids)):
    user_id = Ids[i]
    
    recommended = model.recommend(user_id, sparse_user_item)

    mcats = []
    scores = []
    users = []

# Get mcat names from mcat ids
    for item in recommended:
        idx, score = item
        mcats.append(data.mcat.loc[data.mcat_id == idx].iloc[0])
        #users.append(data.user.loc[data.user_id== idx].iloc[0])
        scores.append(score)

    # Create a dataframe of mcat names and scores
    recommendations = pd.DataFrame({'mcat_id': mcats, 'score': scores})
    recommendations['user_id'] = data.user.loc[data.user_id== Ids[i]].iloc[0]

#print(recommendations)
    
    R1 = R1.append(recommendations)
        
        
print(R1.shape)

#Saving the outputs in the required format
R2 = R1[['customer_id','item_id']]
R2 = R2.sort_values(by='customer_id')
R2.to_csv('user_to_mcat_recommen_ALS_grp_id13.csv',index=False)